In [456]:
import warnings
warnings.filterwarnings('ignore')

In [457]:
import os
import pandas as pd
import math
import numpy as np

Get sorted cases id

In [458]:
import wget
import os.path

if(not os.path.exists('finale.csv')):
    wget.download('https://data.4tu.nl/ndownloader/files/23993303')
    
df_finale = pd.read_csv('finale.csv', index_col=0).reset_index()

df_finale['Complete Timestamp'] = pd.to_datetime(df_finale['Complete Timestamp'])
#sorted_time_cases = df_finale.sort_values('Complete Timestamp')['Case ID'].drop_duplicates().values
sorted_time_cases = df_finale.sort_values('Complete Timestamp')['Case ID'].values

In [459]:
unique_sorted_time_cases = []
for case in sorted_time_cases:
    if(not case in unique_sorted_time_cases):
        unique_sorted_time_cases.append(case)

In [460]:
len(unique_sorted_time_cases)

4580

In [461]:
sorted_time_cases = unique_sorted_time_cases

In [462]:
colapsed = True

if(colapsed):
    df_final = pd.read_csv('finale_time_features_colapsed_prefixed.csv')
    df_final.rename(columns = {'CaseID':'Case ID'}, inplace = True)
else:
    df_final = pd.read_csv('finale_time_features.csv')


Operação para remover espaços na descrição da activity

In [463]:
df_final['Activity'] = df_final['Activity'].str.lower()
df_final['Activity'] = df_final['Activity'].str.replace(' ','-')

In [464]:
df_final.shape

(20404, 8)

- original: (21348)
- colapsed: (20404)

In [465]:
21348 - 20404

944

In [466]:
df_final.columns

Index(['Case ID', 'Activity', 'Duration', 'Duration Float',
       'Time to conclusion', 'Passed Time', 'Step', 'Prefix'],
      dtype='object')

Variables definition

In [467]:
just_closed = False
#mode = 'bukhsh_partition'
mode = 'silva_partition'

Filter for `Closed` process

In [468]:
if(just_closed):
    closed_cases = []

    for name, group in df_final.groupby('Case ID'):
        last_step = group.tail(1)['Activity'].values[0]
        if(last_step == 'Closed'):
            closed_cases.append(group['Case ID'].values[0])

    df_final = df_final[df_final['Case ID'].isin(closed_cases)]     

Activity categorization

In [469]:
df_final['Activity Cat'] = df_final['Activity'].astype('category').cat.codes

Example:

In [470]:
df_final[['Activity Cat', 'Step', 'Duration Float', 'Passed Time', 'Time to conclusion']].head(9)

,Activity Cat,Step,Duration Float,Passed Time,Time to conclusion
0,0,1,0.0000,0.0000,31.0087
1,11,2,16.0084,15.0002,15.0003
2,9,3,0.0001,15.0003,15.0002
3,1,4,15.0002,31.0087,0.0000
4,0,1,0.0000,0.0000,30.9822
5,11,2,5.8750,25.1053,25.1072
6,9,3,0.0019,25.1072,25.1053
7,1,4,25.1053,30.9822,0.0000
8,0,1,14.0421,-0.0000,37.1046


Spliting data (train, test, validation)

In [471]:
from sklearn.model_selection import train_test_split

print(mode)
if (mode == 'bukhsh_partition'):    
    train_cases = pd.read_csv('bukhsh_data/bukhsh_remaining_time_train.csv')['case_id'].unique()
    test_cases = pd.read_csv('bukhsh_data/bukhsh_remaining_time_test.csv')['case_id'].unique()
else:
    train_cases, test_cases  = train_test_split(sorted_time_cases, test_size=0.4, 
                                            shuffle=False, 
                                            random_state=42
                                           )
    
    val_cases, test_cases = train_test_split(test_cases, test_size=0.5, 
                                        shuffle=True,
                                        random_state=42
                                       )

silva_partition


In [472]:
len(train_cases), len(val_cases), len(test_cases)

(2748, 916, 916)

In [473]:
print(
    df_finale[df_finale['Case ID'].isin(train_cases)]['Complete Timestamp'].min(),
    df_finale[df_finale['Case ID'].isin(train_cases)]['Complete Timestamp'].max()
)

2010-01-13 08:40:25 2012-04-07 07:42:34


In [474]:
print(
    df_finale[df_finale['Case ID'].isin(val_cases)]['Complete Timestamp'].min(),
    df_finale[df_finale['Case ID'].isin(val_cases)]['Complete Timestamp'].max()
)

2012-02-10 09:55:04 2014-01-03 10:03:58


In [475]:
print(
    df_finale[df_finale['Case ID'].isin(test_cases)]['Complete Timestamp'].min(),
    df_finale[df_finale['Case ID'].isin(test_cases)]['Complete Timestamp'].max()
)

2012-02-10 14:32:25 2014-01-03 13:20:58


Dataframes creation

In [476]:
df_train = df_final[df_final['Case ID'].isin(train_cases)]
df_test = df_final[df_final['Case ID'].isin(test_cases)]
df_val = df_final[df_final['Case ID'].isin(val_cases)]

Keras tokenizer

In [477]:
from keras.preprocessing.text import Tokenizer

In [478]:
df_prefix_train = df_train['Prefix'].str.split(',', expand = True)
col = df_prefix_train.shape[1]
for i in range(15 - df_prefix_train.shape[1]):
    df_prefix_train.insert(df_prefix_train.shape[1], col+i, None) 
    print(i) 

In [479]:
df_prefix_val = df_val['Prefix'].str.split(',', expand = True)
col = df_prefix_val.shape[1]
for i in range(15 - df_prefix_val.shape[1]):
    df_prefix_val.insert(df_prefix_val.shape[1], col+i, None) 
    print(i)

0
1
2
3
4
5


In [480]:
df_prefix_test = df_test['Prefix'].str.split(',', expand = True)
col = df_prefix_test.shape[1]
for i in range(15 - df_prefix_test.shape[1]):
    df_prefix_test.insert(df_prefix_test.shape[1], col+i, None) 
    print(i)

0
1
2
3
4


In [481]:
df_prefix_train = df_prefix_train.fillna('')
df_prefix_val = df_prefix_val.fillna('')
df_prefix_test = df_prefix_test.fillna('')

In [482]:
#tokenize_mode = 'texts_to_sequences'
#tokenize_mode = 'texts_to_matrix'
tokenize_mode = 'sequences_to_matrix'

In [483]:
kerasTokenizer = Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=",")


train_txt = df_train['Activity'].values
val_txt = df_val['Activity'].values
test_txt = df_test['Activity'].values

"""
train_txt = df_train['Prefix'].values
val_txt = df_val['Prefix'].values
test_txt = df_test['Prefix'].values
"""

"""
train_txt = df_prefix_train.values.tolist()
val_txt = df_prefix_val.values.tolist()
test_txt = df_prefix_test.values.tolist()
"""

kerasTokenizer.fit_on_texts(train_txt)

train_sequences = kerasTokenizer.texts_to_sequences(train_txt)
val_sequences = kerasTokenizer.texts_to_sequences(val_txt)
test_sequences = kerasTokenizer.texts_to_sequences(test_txt)

train_matrix = kerasTokenizer.texts_to_matrix(train_txt, mode='binary')
val_matrix = kerasTokenizer.texts_to_matrix(val_txt, mode='binary')
test_matrix = kerasTokenizer.texts_to_matrix(test_txt, mode='binary')


train_sequences_matrix = kerasTokenizer.sequences_to_matrix(train_sequences, mode='binary')
val_sequences_matrix = kerasTokenizer.sequences_to_matrix(val_sequences, mode='binary')
test_sequences_matrix = kerasTokenizer.sequences_to_matrix(test_sequences, mode='binary')

In [486]:
pd.DataFrame(train_sequences_matrix).fillna(0).astype(int)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12358,0,0,0,1,0,0,0,0,0,0,0,0,0
12359,0,0,0,0,1,0,0,0,0,0,0,0,0
12360,0,1,0,0,0,0,0,0,0,0,0,0,0
12361,0,0,1,0,0,0,0,0,0,0,0,0,0


#### Text to sequences

In [417]:
df_train_sequences = pd.DataFrame(train_sequences).fillna(0).astype(int)
col = df_train_sequences.shape[1]
for i in range(15 - df_train_sequences.shape[1]):
    df_train_sequences.insert(df_train_sequences.shape[1], col+i, 0) 
    print(i)

    
df_test_sequences = pd.DataFrame(test_sequences).fillna(0).astype(int)
col = df_test_sequences.shape[1]
for i in range(15 - df_test_sequences.shape[1]):
    df_test_sequences.insert(df_test_sequences.shape[1], col+i, 0) 
    print(i)
    
df_val_sequences = pd.DataFrame(val_sequences).fillna(0).astype(int)
col = df_val_sequences.shape[1]
for i in range(15 - df_val_sequences.shape[1]):
    df_val_sequences.insert(df_val_sequences.shape[1], col+i, 0) 
    print(i)   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [418]:
#sec = pd.DataFrame([df_train['Activity'].values, matrix]).transpose().rename(columns={0:'0', 1:'1'})['1'].values.tolist()

In [419]:
print(df_train.shape, df_test.shape, df_val.shape)

(12363, 9) (4021, 9) (4020, 9)


Features selection

In [420]:
print(tokenize_mode)

if(tokenize_mode == 'texts_to_sequences'):
    X_train = np.concatenate((df_train[['Step', 'Duration Float', 'Passed Time']].values, 
                              df_train_sequences), axis=1)
    y_train = df_train[['Time to conclusion']].values

    X_test = np.concatenate((df_test[['Step', 'Duration Float', 'Passed Time']].values, 
                             df_test_sequences), axis=1)
    y_test = df_test[['Time to conclusion']].values


    X_val = np.concatenate((df_val[['Step', 'Duration Float', 'Passed Time']].values, 
                            df_val_sequences), axis=1)
    y_val = df_val[['Time to conclusion']].values

if(tokenize_mode == 'texts_to_matrix'):
    X_train = np.concatenate((df_train[['Step', 'Duration Float', 'Passed Time']].values, 
                              train_matrix), axis=1)
    y_train = df_train[['Time to conclusion']].values

    X_test = np.concatenate((df_test[['Step', 'Duration Float', 'Passed Time']].values, 
                             test_matrix), axis=1)
    y_test = df_test[['Time to conclusion']].values


    X_val = np.concatenate((df_val[['Step', 'Duration Float', 'Passed Time']].values, 
                            val_matrix), axis=1)
    y_val = df_val[['Time to conclusion']].values
    
if(tokenize_mode == 'sequences_to_matrix'):
    X_train = np.concatenate((df_train[['Step', 'Duration Float', 'Passed Time']].values, 
                              train_sequences_matrix), axis=1)
    y_train = df_train[['Time to conclusion']].values

    X_test = np.concatenate((df_test[['Step', 'Duration Float', 'Passed Time']].values, 
                             test_sequences_matrix), axis=1)
    y_test = df_test[['Time to conclusion']].values


    X_val = np.concatenate((df_val[['Step', 'Duration Float', 'Passed Time']].values, 
                            val_sequences_matrix), axis=1)
    y_val = df_val[['Time to conclusion']].values        

texts_to_sequences


In [421]:
print(
    pd.DataFrame(X_train).shape,
    pd.DataFrame(X_val).shape,
    pd.DataFrame(X_test).shape
)

(12363, 18) (4020, 18) (4021, 18)


In [422]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.0,0.0000,0.0000,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,16.0084,15.0002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0001,15.0003,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,15.0002,31.0087,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0000,0.0000,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12358,5.0,18.2097,43.9482,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12359,1.0,0.0000,0.0000,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12360,2.0,0.0004,55.6216,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12361,3.0,0.3372,55.9588,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Machine learning models definition

In [423]:
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

regressors = [  
        {'nome' : 'DummyRegressor', 'regressor':  DummyRegressor(strategy='mean')},
        {'nome' : 'LinearRegression', 'regressor': LinearRegression(fit_intercept=False, normalize=True)},
        {'nome' : 'RandomForestRegressor',  'regressor': RandomForestRegressor(n_estimators=20, criterion='mse', max_features='auto', 
                                                                              bootstrap=True, warm_start=True, max_depth=19, random_state=0)},
]

Experiment running

In [424]:
import time
from sklearn.metrics import mean_absolute_error

cols = ['Nome do modelo', 'MAE', 'Val MAE']

data_result = []

for regressor in regressors:    
    t1 = time.time()
    regr = regressor['regressor'].fit(X_train, y_train)
    tf = time.time()
    
    y_pred = regr.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    y_pred_val = regr.predict(X_val)
    val_mae = mean_absolute_error(y_val, y_pred_val)
        
    line = [
        regressor['nome'],
        mae,
        val_mae
    ]
    data_result.append(line)

pd.DataFrame(data_result, columns=cols).sort_values(by='MAE', ascending=True)

,Nome do modelo,MAE,Val MAE
2,RandomForestRegressor,3.789858,3.826536
1,LinearRegression,12.646812,12.717819
0,DummyRegressor,14.889137,14.963008


In [425]:
data_result

[['DummyRegressor', 14.889136625513112, 14.9630078460625],
 ['LinearRegression', 12.64681201802549, 12.717819001920333],
 ['RandomForestRegressor', 3.789858244739419, 3.8265363781487984]]

- original:
    - ['RandomForestRegressor', 3.8718649593112966]
    - ['LinearRegression', 6.7278216084671545]
    - ['DummyRegressor', 14.735706385290072] 
- colapsed:
    - ['RandomForestRegressor', 3.7487898210121866]
    - ['LinearRegression', 6.491372892689034]
    - ['DummyRegressor', 14.926067642377747]

In [426]:
import keras
from keras_self_attention import SeqSelfAttention

In [427]:
from tensorflow import keras

from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import PReLU
from keras.layers import BatchNormalization

from keras.utils.vis_utils import plot_model
from keras.utils.vis_utils import model_to_dot

from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback

In [428]:
from keras.layers.core import Activation
from keras.initializers import Constant

from keras.regularizers import l2

In [429]:
from keras_multi_head import MultiHead
from keras_multi_head import MultiHeadAttention

model = keras.models.Sequential(name='')

"""
model.add(Dense(X_train.shape[1], 
                input_dim=X_train.shape[1], 
                activation='relu', 
                name='Densa'
                ))

model.add(Dropout(0.05, name='Regularizacao'))

model.add(keras.layers.Flatten(name='Nivelamento'))
"""

model.add(LSTM(X_train.shape[1], return_sequences=False, input_shape=(X_train.shape[1], 1)))
##model.add(Bidirectional(LSTM(X_train.shape[1])))

model.add(Dropout(0.05, name='Regularizacao'))

model.add(Dense(1, activation='relu', name='Saida'))

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 18)                1440      
                                                                 
 Regularizacao (Dropout)     (None, 18)                0         
                                                                 
 Saida (Dense)               (None, 1)                 19        
                                                                 
Total params: 1,459
Trainable params: 1,459
Non-trainable params: 0
_________________________________________________________________


In [430]:
nadam = keras.optimizers.Nadam(
    learning_rate=0.01, beta_1=0.9, beta_2=0.99, epsilon=1e-07, name="Nadam")

model.compile(
    optimizer=nadam, 
    loss="mean_absolute_error", 
    #loss=keras.losses.LogCosh(),
    metrics=["mean_absolute_error"]
)

es = EarlyStopping(monitor='mean_absolute_error', 
                   min_delta=1e-07, #min_delta=1e-04, 
                   patience=50, verbose=1)

history = model.fit(X_train, y_train, 
                    epochs=2000,
                    batch_size=X_train.shape[1] * 100,
                    callbacks=[TqdmCallback(verbose=0), es],
                    validation_data=[X_val, y_val],
                    verbose=0)

0epoch [00:00, ?epoch/s]

KeyboardInterrupt: 

In [ ]:
test_predicton = model.predict(X_test)
mae_error = mean_absolute_error(y_test, test_predicton)
print(mae_error)